In [ ]:
#Created By: Rowan Converse
#Date: 2023-03-07
#Purpose: Consensus analysis of USFWS biologist labels of imagery from Bosque del Apache and Maxwell NWR

In [ ]:
#Imports


In [ ]:
#Analysis annotations
path = "/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/Dissertation/1_Chapter/consensus/data/20230307_expertanalysislabels.csv"

#Refined annotations
refinedpath = "/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/CV4Ecology/Prototyping/Data/Labels/train_val_test/20230307_expertconsensuslabels.csv"

In [ ]:
#Refined annotation statistics

In [ ]:
#INDIVIDUAL LABELER METRICS

#Score agreement with consensus class label

#Score %IOU agreement with consensus box 